In [1]:
import sys
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import quant_utils.data_moudle as dm
from dateutil.parser import parse
from quant_utils.performance import periods_performance, Performance
from quant_utils.constant import DB_CONN_JJTG_DATA

In [2]:
query_sql = """
WITH a AS (
	SELECT
		PORTFOLIO_NAME,
		TRADE_DT,
		LOG_RETURN_RATE,
		LOG_BENCHMARK_RET_INNER,
		LOG_FUND_MEDIAN_RET,
		LOG_RETURN_RATE - LOG_BENCHMARK_RET_INNER AS LOG_BENCHMARK_ALPHA,
		LOG_RETURN_RATE - LOG_FUND_MEDIAN_RET AS LOG_PEER_ALPHA 
	FROM
		`portfolio_derivatives_ret` 
	WHERE
		1 = 1 
		AND `PORTFOLIO_NAME` LIKE '%目标盈%' 
		AND TRADE_DT >= '20221230' 
	) SELECT
	a.PORTFOLIO_NAME,
	a.TRADE_DT,
	exp(
	sum( LOG_RETURN_RATE / 100 ) over ( PARTITION BY PORTFOLIO_NAME ORDER BY TRADE_DT ))* 100 - 100 AS '累计收益率(%)',
	exp(
	sum( LOG_BENCHMARK_RET_INNER / 100 ) over ( PARTITION BY PORTFOLIO_NAME ORDER BY TRADE_DT ))* 100 - 100 AS '基准累计收益率(%)',
	exp(
	sum( LOG_FUND_MEDIAN_RET / 100 ) over ( PARTITION BY PORTFOLIO_NAME ORDER BY TRADE_DT ))* 100 - 100 AS '基金中位数累计收益率(%)',
	exp(
	sum( LOG_BENCHMARK_ALPHA / 100 ) over ( PARTITION BY PORTFOLIO_NAME ORDER BY TRADE_DT ))* 100- 100 AS '累计超额_相对基准(%)',
	exp(
	sum( LOG_PEER_ALPHA / 100 ) over ( PARTITION BY PORTFOLIO_NAME ORDER BY TRADE_DT ))* 100- 100 AS '累计超额_相对基金中位数(%)' 
FROM
	a 
ORDER BY
	trade_dt
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)
df1 = df.pivot_table(
    index="TRADE_DT", 
    columns="PORTFOLIO_NAME", 
    values="累计收益率(%)"
)
df2 = df.pivot_table(
    index="TRADE_DT", 
    columns="PORTFOLIO_NAME", 
    values="基金中位数累计收益率(%)"
)
df1 = (df1/100 + 1)
df2 = (df2/100 + 1)
perf_list = []
for col in df1.columns:
    temp_perf = Performance(df1[col].dropna(), df2[col].dropna()).stats().T
    temp_perf["组合名称"] = col
    perf_list.append(temp_perf)
perf_df = pd.concat(perf_list)
perf_df

,起始日期,结束日期,累计收益率,年化收益率,年化波动率,收益波动比,最大回撤,基准累计收益率,基准年化收益率,基准年化波动率,基准收益波动比,基准最大回撤,超额收益,IR,年化收益回撤比,最大回撤日,最大回撤修复,组合名称
0,2023-08-17,2023-10-31,-0.00604,-0.029055,0.013129,-2.21302,0.01035,-0.00891,-0.042619,0.014793,-2.881005,0.01353,0.002869,3.639042,-2.807123,20231023,99999,目标盈定制-内蒙001期
0,2023-06-09,2023-10-31,0.000585,0.001505,0.011037,0.136359,0.010202,0.0,0.0,0.0,NaN,-0.0,0.000585,0.136359,0.147511,20231023,99999,知己目标盈001期
0,2023-06-21,2023-10-31,0.001247,0.003561,0.011155,0.319239,0.010073,0.0,0.0,0.0,NaN,-0.0,0.001247,0.319239,0.353526,20231023,99999,知己目标盈002期
0,2023-07-07,2023-10-31,-0.001638,-0.005236,0.011304,-0.463218,0.010129,-0.005552,-0.017668,0.013994,-1.262541,0.015747,0.003914,2.038719,-0.516959,20231023,99999,知己目标盈003期
0,2023-07-21,2023-10-31,-0.003782,-0.013735,0.01158,-1.186087,0.010176,-0.006972,-0.025215,0.014384,-1.753003,0.015747,0.00319,1.850105,-1.349842,20231023,99999,知己目标盈004期
0,2023-08-04,2023-10-31,-0.005649,-0.023757,0.012135,-1.957767,0.010123,-0.011137,-0.046423,0.014271,-3.252942,0.015747,0.005488,4.601451,-2.346882,20231023,99999,知己目标盈005期
0,2023-08-18,2023-10-31,-0.005647,-0.028301,0.013499,-2.096492,0.01063,-0.007812,-0.038979,0.014795,-2.634551,0.01308,0.002165,2.430698,-2.662391,20231023,99999,知己目标盈006期
0,2023-09-01,2023-10-31,-0.005496,-0.034087,0.013442,-2.535803,0.010187,-0.007535,-0.046485,0.015178,-3.06274,0.01308,0.00204,3.179706,-3.346156,20231023,99999,知己目标盈007期
0,2023-09-15,2023-10-31,-0.002581,-0.02121,0.013765,-1.540929,0.007501,-0.003744,-0.03064,0.015494,-1.977536,0.008971,0.001163,2.784164,-2.82751,20231023,99999,知己目标盈008期
0,2023-09-27,2023-10-31,-0.00198,-0.021048,0.015592,-1.349934,0.007534,-0.003341,-0.035293,0.017494,-2.017449,0.008971,0.001362,3.95584,-2.793533,20231023,99999,知己目标盈009期


In [3]:
df2.columns

Index(['目标盈定制-内蒙001期', '知己目标盈001期', '知己目标盈002期', '知己目标盈003期', '知己目标盈004期',
       '知己目标盈005期', '知己目标盈006期', '知己目标盈007期', '知己目标盈008期', '知己目标盈009期',
       '知己目标盈010期', '知己目标盈灰度测试001期', '知己目标盈灰度测试002期', '知己目标盈灰度测试003期'],
      dtype='object', name='PORTFOLIO_NAME')

In [4]:
df = pd.read_excel("D:/BaiduNetdiskWorkspace/6-定制方案/湖南/885020.WI.xlsx")
df = df.set_index("日期")

In [5]:
perf_list = []
perf_rolling = []
for col in df.columns:
    temp_perf = Performance(df[col].dropna()).stats().T
    temp_rolling = Performance(df[col].dropna()).rolling_ret_stats()
    temp_perf["组合名称"] = col
    perf_list.append(temp_perf)
    temp_rolling["组合名称"] = col
    perf_rolling.append(temp_rolling)
perf_df = pd.concat(perf_list)
perf_rolling_df = pd.concat(perf_rolling)
perf_df

,起始日期,结束日期,累计收益率,年化收益率,年化波动率,收益波动比,最大回撤,年化收益回撤比,最大回撤日,最大回撤修复,组合名称
0,2018-12-28,2023-09-28,0.619851,0.107057,0.122495,0.873972,0.136996,0.781461,20220426,99999,万得价值风格型基金指数
0,2018-12-28,2023-09-28,0.596725,0.103706,0.19679,0.526985,0.355556,0.291671,20230921,99999,偏股基金
0,2018-12-28,2023-09-28,0.668982,0.114054,0.193093,0.590669,0.311367,0.366301,20220426,99999,主动权益-指数增强-中证500
0,2018-12-28,2023-09-28,0.365336,0.067866,0.207918,0.326406,0.315704,0.214967,20220426,99999,中证500
0,2018-12-28,2023-09-28,0.203413,0.039816,0.029525,1.348529,0.048926,0.813787,20190312,118,超额
0,2018-12-28,2023-09-28,0.636218,0.109407,0.132996,0.822631,0.174655,0.626416,20220426,99999,价值80+指增20
0,2018-12-28,2023-09-28,0.643209,0.110404,0.139147,0.793438,0.192942,0.572214,20220426,99999,价值70+指增30
0,2018-12-28,2023-09-28,1.325526,0.194772,0.170263,1.143945,0.226107,0.861414,20220426,99999,知己-权益价值组合
0,2018-12-28,2023-09-28,0.954479,0.151773,0.19158,0.792218,0.296698,0.511542,20220426,99999,知己-中证500指增
0,2018-12-28,2023-09-28,1.249389,0.186415,0.172884,1.078264,0.232298,0.802481,20220426,99999,混合


In [6]:
perf_rolling_df

,7天,1个月,2个月,3个月,6个月,9个月,1年,2年,3年,4年,5年,组合名称
收益率25分位数,-0.82%,-1.03%,-1.00%,-0.46%,0.57%,0.86%,0.62%,4.25%,26.28%,38.68%,nan%,万得价值风格型基金指数
收益率中位数,0.36%,0.78%,1.32%,1.94%,3.53%,5.82%,7.20%,23.98%,32.79%,41.94%,nan%,万得价值风格型基金指数
收益率75分位数,1.49%,2.75%,3.60%,4.15%,7.73%,11.78%,21.17%,36.10%,37.83%,47.07%,nan%,万得价值风格型基金指数
胜率,58.20%,61.96%,64.96%,70.81%,78.80%,77.80%,77.98%,93.02%,100.00%,100.00%,nan%,万得价值风格型基金指数
收益率25分位数,-1.59%,-2.61%,-4.39%,-4.53%,-5.10%,-9.17%,-13.13%,-14.95%,15.22%,33.48%,nan%,偏股基金
收益率中位数,0.36%,0.85%,1.14%,1.46%,3.78%,8.22%,11.16%,16.52%,30.01%,39.68%,nan%,偏股基金
收益率75分位数,2.38%,4.21%,6.92%,8.85%,14.29%,22.68%,40.44%,66.45%,51.07%,50.06%,nan%,偏股基金
胜率,54.71%,56.49%,55.80%,56.82%,59.83%,57.66%,55.42%,59.44%,83.83%,100.00%,nan%,偏股基金
收益率25分位数,-1.39%,-2.30%,-3.41%,-3.30%,-2.73%,-4.12%,-5.37%,1.49%,27.80%,39.33%,nan%,主动权益-指数增强-中证500
收益率中位数,0.38%,0.61%,0.82%,1.14%,3.96%,7.77%,12.51%,25.23%,38.75%,46.41%,nan%,主动权益-指数增强-中证500


In [7]:
df = pd.read_excel("d:/产品池.xlsx")
df["产品代码"] = df["产品代码"].apply(lambda s: str(s).upper())
df["产品代码"] = df["产品代码"].str.replace(".OF", "")
df["产品代码"] = df["产品代码"].str.replace("/", ";")
df["产品代码"] = df["产品代码"].str.replace(",", ";")
df["产品代码"] = df["产品代码"].str.replace("，", ";")
df["产品代码"] = df["产品代码"].str.replace("；", ";")
df["产品代码"] = df["产品代码"].str.replace("、", ";")
df["产品代码"] = df["产品代码"].str.replace("、", ";")

FileNotFoundError: [Errno 2] No such file or directory: 'd:/产品池.xlsx'

In [ ]:
df["产品代码"] = df["产品代码"].str.split(";")

In [ ]:
df = df.explode('产品代码')
df["产品代码"] = df["产品代码"].str.rjust(6,"0")

In [ ]:
df.to_excel("d:/产品池a.xlsx")